In [2]:
import sys
from pathlib import Path, PureWindowsPath
current_folder = PureWindowsPath(globals()['_dh'][0]).as_posix()
path_root = "/".join(current_folder.split("/")[:-3]) #remonter au dossier root du projet
os.chdir(path_root)
print(os.getcwd())
sys.path.append(os.getcwd() + "/sources/classification/")

c:\Users\eupho\OneDrive\Documents\perso\projets\classification_texte_bapteme_philo


In [3]:
import os
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from pathlib import Path, PureWindowsPath
from lib_classification import *
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.min_rows', 5)
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_colwidth', None) #afficher texte entier dans dataframe

In [4]:
path = PureWindowsPath(os.getcwd() + "/data/input/data_middle_age_epistemology.parquet")
path = path.as_posix() #convertir en path linux (convertir les \\ en /)
corpus = pd.read_parquet(path) #engine="fastparquet"
corpus = get_balanced_binary_dataset(corpus, class_col_name="category")

# print(corpus["category_bin"].value_counts())
# print("presence de doublons ?")
# print(corpus.id.duplicated().any())
# print(corpus.index.duplicated().any())
# corpus

In [20]:
features_col_names = "message_preprocessed" 
# class_col_name = "category"
class_col_name = "category_bin" 
id_col_name = "id"
X_train, X_test, y_train, y_test, indices_train, indices_test = get_train_and_test(corpus, features_col_names, class_col_name, id_col_name)

In [21]:
X_train_tfidf, X_test_tfidf = apply_tfidf_to_train_and_test(X_train, X_test)

In [22]:
scoring = ['accuracy', 'f1_macro']
num_iter = 2 #nombre de repetitions de la k-fold cross validation entiere
k = 10 #k de la k-fold cross validation
do_cross_validation(X_train_tfidf, y_train, scoring, num_iter, k)

[('test_accuracy', 0.9164), ('test_f1_macro', 0.9155)] (LR)
[('test_accuracy', 0.9457), ('test_f1_macro', 0.9456)] (AdaBoostClassifier)
[('test_accuracy', 0.9342), ('test_f1_macro', 0.934)] (KNN)
[('test_accuracy', 0.9435), ('test_f1_macro', 0.9433)] (RandomForest)
[('test_accuracy', 0.9478), ('test_f1_macro', 0.9476)] (SGDClassifier)
[('test_accuracy', 0.9133), ('test_f1_macro', 0.9125)] (SVM)
[('test_accuracy', 0.9445), ('test_f1_macro', 0.9443)] (DecisionTreeClassifier)


In [23]:
k = 10
# kfold = StratifiedKFold(n_splits=k, random_state=None, shuffle=True) #n_splits = k de la k-fold cv, le nombre de fold
kfold = RepeatedStratifiedKFold(n_splits=k, n_repeats=2, random_state=None)
scoring = 'accuracy'
cv_param = kfold
num_experiences = 10
train_sizes = np.linspace(0.1, 1.0, num_experiences)
# train_sizes = np.linspace(0.001, 1.0, num_experiences)
n_jobs = -1

scorings = ['accuracy', 'precision', 'recall', 'f1']
for scoring in scorings:
        get_learning_curve(model, X_train_tfidf, y_train, cv_param, scoring, train_sizes, n_jobs=-1)